<a href="https://colab.research.google.com/github/nisanuro/CNG483-Project1/blob/master/CNG483_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>